In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

In [3]:
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorbord_root_log_dir: Path
    checkpoint_model_filepath: Path


In [4]:
from deepClassifier.utils import read_yaml, create_directories
from deepClassifier.constants import *

In [5]:
class Configuration:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifact_root])
    
    def get_prepare_callback_config(self) ->PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckptl_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckptl_dir), Path(config.tensorbord_root_log_dir)])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorbord_root_log_dir=Path(config.tensorbord_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config


In [6]:
from pathlib import Path
from zipfile import ZipFile
import os
from urllib import request
from tqdm import tqdm
import pandas
import tensorflow as tf
import time

class PrepareCallBack:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config=config
    
    @property
    def _create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir=os.path.join(
            self.config.tensorbord_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [7]:
try:
    config=Configuration()
    prepare_callback_config=config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallBack(config=prepare_callback_config)
    prepare_callbacks.get_ckpt_callbacks()
except Exception as e:
    raise e

[2022-09-20 22:51:48,273: INFO: common]: yaml file: configs\confIg.yaml loaded successfully
[2022-09-20 22:51:48,283: INFO: common]: yaml file: params.yaml loaded successfully
[2022-09-20 22:51:48,284: INFO: common]: created directory at: artifacts
[2022-09-20 22:51:48,287: INFO: common]: created directory at: artifacts\prepare_callbacks\checkpoint_dir
[2022-09-20 22:51:48,288: INFO: common]: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir
